In [1]:
import os
from dotenv import load_dotenv, find_dotenv
import pandas as pd
from openai import OpenAI

In [2]:
load_dotenv(find_dotenv(), override=True)

True

In [3]:
CSV_QUESTIONS = "05.0_questions.csv"
CSV_GROUND_TRUTH = "05.0_ground_truths.csv"
CSV_CONTEXTS = "05.3_selected_contexts.csv"  # RAG contexts
CSV_LLM_ANSWERS = "05.5_answers.csv"  # LLM answers
CSV_EVALS_NO_GROUND_TRUTH = "05.6_evals_no_ground_truths.csv"  # All bar ground truths
CSV_COMPLETE = "05.7_complete_evals_dataset.csv"  # ALL

In [4]:
def get_llm_client(llm_choice):
    if llm_choice == "GROQ":
        client = OpenAI(
            base_url="https://api.groq.com/openai/v1",
            api_key=os.environ.get("GROQ_API_KEY"),
        )
        return client
    elif llm_choice == "OPENAI":
        load_dotenv()  # load environment variables from .env fil
        client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
        return client
    else:
        raise ValueError("Invalid LLM choice. Please choose 'GROQ' or 'OPENAI'.")

In [5]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

LLM_CHOICE = "OPENAI"
LLM_CHOICE = "GROQ"

if OPENAI_API_KEY:
    print(f"OPENAI_API_KEY exists and begins {OPENAI_API_KEY[:14]}...")
else:
    print("OPENAI_API_KEY not set")

if GROQ_API_KEY:
    print(f"GROQ_API_KEY exists and begins {GROQ_API_KEY[:14]}...")
else:
    print("GROQ_API_KEY not set")


client = get_llm_client(LLM_CHOICE)
if LLM_CHOICE == "GROQ":
    MODEL = "llama-3.3-70b-versatile"
else:
    MODEL = "gpt-4o-mini"

print(f"LLM_CHOICE: {LLM_CHOICE} - MODEL: {MODEL}")

OPENAI_API_KEY exists and begins sk-proj-vIt-L1...
GROQ_API_KEY exists and begins gsk_0yKDCuUXkz...
LLM_CHOICE: GROQ - MODEL: llama-3.3-70b-versatile


In [6]:
list_questions = []

with open(CSV_QUESTIONS, "r") as f:
    for line in f:
        list_questions.append(line.strip())

In [7]:
print(list_questions)

['What was the climate change about last year?', 'Why was food price volatility so high?', 'What does CRISPR-Cas9 enable?', 'Optimise self driving?']


In [8]:
df_contexts = pd.read_csv(CSV_CONTEXTS, sep="|")
df_contexts.head()

,id,question,contexts,metadata
0,69b8a036-74e0-440d-9d5e-8a42cd0d4efa,What was the climate change about last year?,Climate change poses significant risks to glob...,"{'keywords_str': 'climate change, food securit..."
1,69b8a036-74e0-440d-9d5e-8a42cd0d4efa,Why was food price volatility so high?,Climate change poses significant risks to glob...,"{'source': 'journal_article', 'author': 'Maria..."
2,8a2fb5a6-9f44-4cc3-ad4d-6551404a0e15,What does CRISPR-Cas9 enable?,Genetic editing techniques show promise for tr...,"{'keywords_str': 'genetic editing, CRISPR, med..."
3,cde31609-587d-4b68-a4d7-5f936c58b3bd,Optimise self driving?,Urban planning strategies are adapting to acco...,"{'keywords_str': 'urban planning, autonomous v..."


In [9]:
list_answers = []

for i, question in enumerate(list_questions):
    context = df_contexts.loc[i, "contexts"]
    # print(i, question, context)
    prompts = [
        {
            "role": "system",
            "content": f"""You are a RAG specialist. You are given a question from the user and the most relevant context will be provided in <context></context>. You will answer the question based on the context. The answer should be a short and concise answer: 
            <context>
            {context}
            </context>
            
            """,
        },
        {"role": "user", "content": question},
    ]
    # get llm answer
    response = client.chat.completions.create(
        model=MODEL, messages=prompts, temperature=0.7
    )
    # print(response.choices[0].message.content)
    answer = response.choices[0].message.content
    list_answers.append(answer)
    print(f"Question: {question}")
    print(f"Answer: {answer}")
    print(f"Context: {context}")
    print("-" * 100)

Question: What was the climate change about last year?
Answer: According to the context, climate change last year, and in general, posed significant risks to global food security, including rising temperatures, altered precipitation patterns, and increased frequency of extreme weather events, which disrupted traditional farming practices and crop yields.
Context: Climate change poses significant risks to global food security and agricultural systems, threatening the livelihoods of billions of people worldwide and challenging the fundamental stability of food production networks. Rising temperatures, altered precipitation patterns, and increased frequency of extreme weather events are disrupting traditional farming practices and crop yields across different geographic regions. Agricultural communities face unprecedented challenges as changing climate conditions affect soil quality, water availability, and pest populations, forcing farmers to adapt their cultivation methods and crop sele

In [10]:
# convert list_answers to pandas dataframe
df_answers = pd.DataFrame({"answer": list_answers})
df_answers.to_csv(CSV_LLM_ANSWERS, index=False)
df_answers

,answer
0,"According to the context, climate change last ..."
1,Food price volatility was high due to supply c...
2,CRISPR-Cas9 enables precise modifications to D...
3,"To optimize self-driving technology, city plan..."


In [11]:
# merge df_contexts and df_answers
df_merged = pd.concat([df_contexts, df_answers], axis=1)
df_merged

,id,question,contexts,metadata,answer
0,69b8a036-74e0-440d-9d5e-8a42cd0d4efa,What was the climate change about last year?,Climate change poses significant risks to glob...,"{'keywords_str': 'climate change, food securit...","According to the context, climate change last ..."
1,69b8a036-74e0-440d-9d5e-8a42cd0d4efa,Why was food price volatility so high?,Climate change poses significant risks to glob...,"{'source': 'journal_article', 'author': 'Maria...",Food price volatility was high due to supply c...
2,8a2fb5a6-9f44-4cc3-ad4d-6551404a0e15,What does CRISPR-Cas9 enable?,Genetic editing techniques show promise for tr...,"{'keywords_str': 'genetic editing, CRISPR, med...",CRISPR-Cas9 enables precise modifications to D...
3,cde31609-587d-4b68-a4d7-5f936c58b3bd,Optimise self driving?,Urban planning strategies are adapting to acco...,"{'keywords_str': 'urban planning, autonomous v...","To optimize self-driving technology, city plan..."


In [12]:
# save df_merged to csv
df_merged.to_csv(CSV_EVALS_NO_GROUND_TRUTH, index=False)

In [13]:
df_ground_truth = pd.read_csv(CSV_GROUND_TRUTH, sep="|")
df_ground_truth.head()

,ground_truth
0,What was the climate change about last year?
1,Why was food price volatility so high?
2,What does CRISPR-Cas9 enable?
3,Optimise self driving?


In [14]:
# merge df_contexts and df_answers
df_complete = pd.concat([df_merged, df_ground_truth], axis=1)
df_complete

,id,question,contexts,metadata,answer,ground_truth
0,69b8a036-74e0-440d-9d5e-8a42cd0d4efa,What was the climate change about last year?,Climate change poses significant risks to glob...,"{'keywords_str': 'climate change, food securit...","According to the context, climate change last ...",What was the climate change about last year?
1,69b8a036-74e0-440d-9d5e-8a42cd0d4efa,Why was food price volatility so high?,Climate change poses significant risks to glob...,"{'source': 'journal_article', 'author': 'Maria...",Food price volatility was high due to supply c...,Why was food price volatility so high?
2,8a2fb5a6-9f44-4cc3-ad4d-6551404a0e15,What does CRISPR-Cas9 enable?,Genetic editing techniques show promise for tr...,"{'keywords_str': 'genetic editing, CRISPR, med...",CRISPR-Cas9 enables precise modifications to D...,What does CRISPR-Cas9 enable?
3,cde31609-587d-4b68-a4d7-5f936c58b3bd,Optimise self driving?,Urban planning strategies are adapting to acco...,"{'keywords_str': 'urban planning, autonomous v...","To optimize self-driving technology, city plan...",Optimise self driving?


In [15]:
df_complete.to_csv(CSV_COMPLETE, index=False)

### We can now perform the evaluations